# Project group 30

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.set_printoptions(precision = 4, suppress = True)

In [2]:
#pip install pycountry_convert
from pycountry_convert import country_name_to_country_alpha2

In [3]:
population = 'population.csv'
df_population= pd.read_csv(population)
medals = 'medals.csv'
df_medals= pd.read_csv(medals)
cpi = 'cpi.csv'
df_cpi= pd.read_csv(cpi)

#df_population

In [6]:
import pycountry
import pycountry_convert as pcc
from fuzzywuzzy import fuzz, process

# Create a list of all unique countries in the dataframe
countries = df_medals['Country'].unique().tolist()
#print(countries)

#Create a list of all standard country names
countries_standard = [country.name for country in pycountry.countries]
#print(countries_standard)

for i in range(len(df_medals)):
    temp = df_medals['Country'][i]
    country = 'c'
    try:
        #print(pycountry.countries.lookup(temp))
        country = pycountry.countries.lookup(temp)
        #print(country)
    except LookupError as e:
        print('a')
        #continue
    if country == 'c':
        #print('k')
        try:
            match = process.extractOne(str(temp), countries_standard, scorer=fuzz.token_sort_ratio)
            #print(match)
            #print('kk')
            if match[1] > 74:
                #print(match[0])
                c = pycountry.countries.get(name=match[0])
                country_code=c.alpha_3
                #country_code = pcc.country_alpha3_to_country_name(match[0])
                df_medals.loc[i, 'Country'] = country_code
            else:
                df_medals.loc[i, 'Country'] = None
        except ValueError as e:
            df_medals.loc[i, 'Country'] = None
    else:
        df_medals.loc[i, 'Country'] = country.alpha_3

#manually converting few instances
#Kosovo doesnt have ISO3
df_medals.at[3, 'Country'] = 'GBR'
df_medals.at[4, 'Country'] = 'RUS'
df_medals.at[33, 'Country'] = 'TWN'

#print(countries)
df_medals['Country']

a


0     USA
1     CHN
2     JPN
3     GBR
4     RUS
     ... 
88    GHA
89    GRD
90    KWT
91    MDA
92    SYR
Name: Country, Length: 93, dtype: object

In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df_medals)

   Country  Gold Medal  Silver Medal  Bronze Medal  Total  Rank By Total
0      USA          39            41            33    113              1
1      CHN          38            32            18     88              2
2      JPN          27            14            17     58              5
3      GBR          22            21            22     65              4
4      RUS          20            28            23     71              3
5      AUS          17             7            22     46              6
6      NLD          10            12            14     36              9
7      FRA          10            12            11     33             10
8      DEU          10            11            16     37              8
9      ITA          10            10            20     40              7
10     CAN           7             6            11     24             11
11     BRA           7             6             8     21             12
12     NZL           7             6             7 

In [20]:
df_merged1 = pd.merge(df_medals, df_population, left_on='Country', right_on='CCA3')
df_merged2 = pd.merge(df_merged1, df_cpi, left_on='Country', right_on='ISO3')
df_merged2
df = df_merged2[['Country_x','Total','2020 Population', 'CPI score 2019']]
df

,Country_x,Total,2020 Population,CPI score 2019
0,USA,113,335942003,69
1,CHN,88,1424929781,41
2,JPN,58,125244761,73
3,GBR,65,67059474,77
4,RUS,71,145617329,28
...,...,...,...,...
83,GHA,1,32180401,41
84,GRD,1,123663,53
85,KWT,1,4360444,40
86,MDA,1,3084847,32


In [17]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df)

    Country_x CCA3 ISO3  Total  2020 Population  CPI score 2019
0         USA  USA  USA  113.0     3.359420e+08            69.0
1         CHN  CHN  CHN   88.0     1.424930e+09            41.0
2         JPN  JPN  JPN   58.0     1.252448e+08            73.0
3         GBR  GBR  GBR   65.0     6.705947e+07            77.0
4         RUS  RUS  RUS   71.0     1.456173e+08            28.0
5         AUS  AUS  AUS   46.0     2.567005e+07            77.0
6         NLD  NLD  NLD   36.0     1.743456e+07            82.0
7         FRA  FRA  FRA   33.0     6.448005e+07            69.0
8         DEU  DEU  DEU   37.0     8.332899e+07            80.0
9         ITA  ITA  ITA   40.0     5.950058e+07            53.0
10        CAN  CAN  CAN   24.0     3.788870e+07            77.0
11        BRA  BRA  BRA   21.0     2.131963e+08            35.0
12        NZL  NZL  NZL   20.0     5.061133e+06            87.0
13        CUB  CUB  CUB   15.0     1.130070e+07            48.0
14        HUN  HUN  HUN   20.0     9.750